### Sending to Generators

So far we have seen how yield can produce values
- use iteration to get the produced values -> next()

After a value is yielded, the generator is suspended

How about sending data to the generator upon resumption?

Yes, this was an enhancement to generators in Python 2.5

#### Sending data to a generator

yield is actually an expression
- it can yield a value (like we have seen before) -> yield 'hello'
- it can also receive values -> it is used just like an expression would -> received = yield

We can combine both -> recieved = yield 'hello'
- this works, but is confusing
- use sparingly

Whats happening?

In [13]:
def gen_echo():
    while True:
        received = yield
        print('You said: ', received)

In [14]:
echo = gen_echo() # created

In [15]:
next(echo) # suspended -> Python has just yielded (None) and the generator is suspended at the yield

We can resume and send data to the generator at the same time using send()

In [16]:
echo.send('hello') #This resumes the generator, and sends data (so it calls next(echo))

You said:  hello


So in above, yield is returning 'hello'

And the generator resumes running exactly at the yield, the yield expression evaluates to the just received data, then the assignmed to received is made

#### What's happening???

when we have code like this:

In [ ]:
received = yield 'python'

The generator gets suspended **right** before the yield.

'python' is yielded and control is returned to the caller

caller sends data to generator: g.send('hello')

Then the generator resumes, exactly where it was suspended

So *received* no longer evaluates to *yield 'python'*

But rather to *'hello'*

ie

In [ ]:
received = 'hello'

So 'hello' is the result of the yield expression. and 'hello' is assigned to received

And the generator continues running until the next yield or return

#### Priming the generator

Notice that we can only send data if the generator is suspended at a yield

So we cannot send data to a generator that is in  a created state - it must be in a suspended state

So if we do

In [17]:
echo = gen_echo()

We cannot send data as it is in a created state

In [18]:
echo.send('hello')

TypeError: can't send non-None value to a just-started generator

So to prime it we must first call next(gen)

In [19]:
next(echo)

The generator is now suspended...yes a value has been yielded, and we can choose to just ignore it

In the above example, None has been yielded

In [20]:
echo.send('hello')

You said:  hello


Since it was in a suspended state, we could send data.

SO! Don't forget to prime a generator before sending values to it!
- generator MUST be **suspended** to recieve data
- always use next() to prime

In a bit, we'll look at how we can "automatically" prime a generator using a decorator

#### Using yield...

- Used for producing data -> yield 'Python'
- Used for receiving data -> a = yield (technically this produces **None**  
Be careful mixing the two usages in your code
- difficult to understand
- sometimes useful
- often not needed

#### Code Examples

In [35]:
def echo():
    while True:
        received = yield
        print('You said:', received)

In [36]:
e = echo()

In [37]:
from inspect import getgeneratorstate

In [38]:
getgeneratorstate(e)

'GEN_CREATED'

In [39]:
next(e)

In [40]:
getgeneratorstate(e)

'GEN_SUSPENDED'

In [41]:
e.send('it')

You said: it


In [42]:
e.send('send')

You said: send


In [43]:
def squares(n):
    for i in range(n):
        yield i**2

In [44]:
sq = squares(5)

In [45]:
next(sq)

0

In [46]:
sq.send('python')

1

In [47]:
def echo():
    while True:
        received = yield
        print('You said:', received)

In [48]:
e = echo()

In [49]:
e.send(None)

In [50]:
getgeneratorstate(e)

'GEN_SUSPENDED'

In [51]:
e.send('hello')

You said: hello


In [52]:
e = echo()

In [53]:
e.send('failure')

TypeError: can't send non-None value to a just-started generator

In [55]:
def squares(n):
    for i in range(n):
        received = yield i**2
        print(received)

In [56]:
sq = squares(5)

In [57]:
next(sq)

0

In [58]:
next(sq)

None


1

In [59]:
sq.send('python')

python


4

In [60]:
def echo(max_times):
    for _ in range(max_times):
        received = yield
        print('You said:', received)
    print("That's all folks!")

In [61]:
e = echo(3)

In [62]:
next(e)

In [63]:
e.send('python')

You said: python


In [64]:
e.send('IS')

You said: IS


In [65]:
e.send('awesome')

You said: awesome
That's all folks!


StopIteration: 

In [66]:
def averager():
    total = 0
    count = 0
    def inner(value):
        nonlocal total
        nonlocal count
        total += value
        count += 1
        return total / count
    return inner

In [68]:
def running_averages(iterable):
    avg = averager()
    for value in iterable:
        running_average = avg(value)
        print(running_average)

In [69]:
running_averages([1, 2, 3, 4, 5])

1.0
1.5
2.0
2.5
3.0


In [70]:
def running_averager():
    total = 0
    count = 0
    running_average = None
    while True:
        value = yield running_average
        total += value
        count += 1
        running_average = total / count

In [71]:
def running_averages(iterable):
    averager = running_averager()
    next(averager)
    for value in iterable:
        running_average = averager.send(value)
        print(running_average)

In [72]:
running_averages([1, 2, 3, 4])

1.0
1.5
2.0
2.5
